In [1]:
#importing earth engine
import ee


In [2]:
#Authenticating the flow
ee.Authenticate()

Enter verification code: 4/1AfJohXnMSndGPfvRfC7N6M4q7a6kILv7VmfjxFFOTCDVXewjYS5bNiER9Bo

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [12]:
#define region of interest
region= ee.Geometry.Rectangle(31.56, -26.24, 31.78,-26.09)

#loading Landsat data
image= ee.ImageCollection('LANDSAT/LC08/C01/T1')\
.filterDate('2017-01-01', '2017-12-31')\
.filterBounds(region)\
.sort('CLOUD_COVER', True)\
.first()

#Making the training data
training= image.sample(**{
    'region': region,
    'scale': 30,
    'numPixels': 5000  
})
#initiating the clusterer and training it
clusterer= ee.Clusterer.wekaKMeans(15).train(training)

#cluster the input using trained clusterer
result= image.cluster(clusterer)

In [5]:
#import folium library
import folium


In [6]:
#defining a method for displaying GEE tiles to folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
     map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
     folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

In [7]:
# adding EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [8]:
#creating a folium map object
my_map= folium.Map(location=[-26.1694, 31.6632], zoom_start= 11)

In [9]:
#adding the layer to map object
my_map.add_ee_layer(result.clip(region).randomVisualizer(),{}, 'clusters')

In [10]:
#adding layer control panel to map
my_map.add_child(folium.LayerControl())

In [11]:
#displaying the map
display(my_map)